In [1]:
import os
import time
import pandas as pd
from tqdm.auto import tqdm
from pandas import DataFrame
from datasets import load_dataset
import httpx


# Import OpenAI client and initialize with your API key.
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

/Users/phani/Repo/itsmephani/open-ai/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tools = [
    {
        "type": "function",
        "name": "get_current_weather",
        "description": "Get current weather for the provided city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                  "type": "string",
                  "description": "City name e.g. Hyderabad, India",
                },
            },
            "required": ["city"],
            "additionalProperties": False,
        },
        "strict": True,
    },
]

In [ ]:
from tool import get_current_weather
get_current_weather("Hyderabad, India")


{'success': True,
 'location': 'Hyderabad',
 'temperature_C': '22',
 'temperature_F': '72',
 'condition': 'Mist',
 'humidity': '100',
 'wind_kph': '9',
 'raw': {'current_condition': [{'FeelsLikeC': '25',
    'FeelsLikeF': '76',
    'cloudcover': '50',
    'humidity': '100',
    'localObsDateTime': '2025-10-09 05:02 AM',
    'observation_time': '11:32 PM',
    'precipInches': '0.0',
    'precipMM': '0.0',
    'pressure': '1014',
    'pressureInches': '30',
    'temp_C': '22',
    'temp_F': '72',
    'uvIndex': '0',
    'visibility': '4',
    'visibilityMiles': '2',
    'weatherCode': '143',
    'weatherDesc': [{'value': 'Mist'}],
    'weatherIconUrl': [{'value': ''}],
    'winddir16Point': 'N',
    'winddirDegree': '353',
    'windspeedKmph': '9',
    'windspeedMiles': '6'}],
  'nearest_area': [{'areaName': [{'value': 'Hyderabad'}],
    'country': [{'value': 'India'}],
    'latitude': '17.375',
    'longitude': '78.474',
    'population': '3597816',
    'region': [{'value': 'Telangana'}

In [4]:
def call_function(name, args):
  if name == "get_current_weather":
    return get_current_weather(**args)
  
  raise ValueError(f"Function {name} not found")

In [6]:
history = [
    {
        "role": "user",
        "content": "What's the weather like in Hyderabad, India?"
    }
]
response = client.responses.create(
  model="gpt-5-nano",
  tools=[],
  input=history,
)
print(f"1. {response.output_text}")
history.append({"role": "assistant", "content": response.output_text})
history.append({"role": "user", "content": "What is 2*3?"})

response = client.responses.create(
  model="gpt-5-nano",
  tools=[],
  previous_response_id=response.id,
  input=history,
)
print(f"2. {response.output_text}")
history.append({"role": "assistant", "content": response.output_text})
history.append({"role": "user", "content": "What are the tourist places"})

response = client.responses.create(
  model="gpt-5-nano",
  tools=[],
  previous_response_id=response.id,
  input=history,
)
print(f"3. {response.output_text}")
print("-------------------------------Full history:")
print(history)

1. I don’t have live weather data here. If you want the current temperature and a short-term forecast, I can fetch it for you if you’d like to enable internet access or tell me a source you trust.

In the meantime, here’s the typical climate for Hyderabad, India (Telangana) around this time of year (October/early November):

- Season: Post-monsoon, hot and humid but gradually easing from the summer peak.
- Daytime highs: roughly 33–34°C (91–93°F).
- Nighttime lows: about 20–22°C (68–72°F).
- Humidity: generally high, but a bit lower than in the peak monsoon months.
- Rain: usually low; isolated showers are possible but most days are dry.
- Sunshine: plenty of sunny hours, with some partly cloudy days.
- What to wear: light, breathable clothing (cotton), sunscreen, and stay hydrated. A light jacket or umbrella can be handy for cooler evenings or occasional showers.
- Best time to visit: October–February, when daytime temps are more comfortable (around 22–28°C).

Want me to pull the curr

In [3]:
# FAISS

from langchain_openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever

loader = PyPDFLoader("docs/ai_report_2025.pdf")
documents = loader.load()
print(f"Loaded {len(documents)} documents from the PDF.")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
print(f"Split into {len(docs)} chunks of text.")



Loaded 26 documents from the PDF.
Split into 43 chunks of text.


In [21]:
print(len(docs[0].page_content))

158


In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [5]:
# Build a FAISS vector store from the split documents and embeddings.
# Using FAISS.from_documents lets the library compute embeddings and create
# the correct index under the hood, avoiding direct faiss.Index* usage.
library = FAISS.from_documents(docs[:10], embeddings)
# Persist the index locally
library.save_local("faiss_index")

# To load this index in a fresh session, run:
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# library = FAISS.load_local("faiss_index", embeddings)

In [9]:
library = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [24]:
query = 'what are RESEARCH METHODOLOGY AND LIMITATIONS'

In [25]:
library.similarity_search(query, k=2)

[Document(id='32b961eb-f0aa-4218-81ae-d49c05a9f53e', metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-07-13T21:18:19-07:00', 'msip_label_87867195-f2b8-4ac2-b0b6-6bb73cb33afc_siteid': '72f988bf-86f1-41af-91ab-2d7cd011db47', 'msip_label_87867195-f2b8-4ac2-b0b6-6bb73cb33afc_method': 'Privileged', 'msip_label_87867195-f2b8-4ac2-b0b6-6bb73cb33afc_enabled': 'True', 'author': 'Aditya Challapally', 'moddate': '2025-07-13T21:18:19-07:00', 'source': 'docs/ai_report_2025.pdf', 'total_pages': 26, 'page': 23, 'page_label': '24'}, page_content='pg. 24 \n \nat scale. Our research focuses on translating AI capabilities into measurable business \noutcomes across enterprise and mid-market organizations. We acknowledge the generous \nparticipation of executives who shared their implementation experiences and insights. \n8.2 RESEARCH METHODOLOGY AND LIMITATIONS \nMethodology: 52 structured interviews across enterprise stakehol

In [27]:
results = library.similarity_search_with_score(query, k=2)

In [31]:
results[1][1]

np.float32(1.1849279)

In [ ]:
library.save_local("faiss_index")

In [11]:
import uuid

uuid.uuid4().hex

'0187b45c22b74d4da49afcb1065bac68'

In [15]:
input_store = {"a": 2}

input_store.setdefault("b", 1)


print(input_store.get("b"))

1


In [16]:
import json

json.dumps(input_store)

'{"a": 2, "b": 1}'

In [ ]:
"="*40

'========================================'